In [2]:
import os
import sys
sys.path.insert(0, "../..")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, StorageCreds, PAR, Authorisation

## Start with uploading and processing of the data

In [21]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/73.60.ef.41
(please check that this page displays the message 'fluffy sheep sit happily')


Check we're logged in

In [22]:
user.wait_for_login()

True

### Upload a file for processing


In [28]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

In [29]:
creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

In [32]:
hugs = Service(service_url="%s/hugs" % base_url)
par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [33]:
hugs

Service(url=hugs.acquire-aaai.com/t/hugs, uid=a0-a7)

In [34]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}

response = hugs.call_function(function="process", args=args)

ImportError: Error calling 'process' on 'https://hugs.acquire-aaai.com/t/hugs': Please clone Acquire into the directory /function/acquire

{'user_uid': 'f5c4f9a8-30db-4916-af9d-bf703451a260',
 'session_uid': 'ed67eec5-5c19-4f99-8ea0-55600439b19f',
 'identity_url': 'hugs.acquire-aaai.com/t/identity',
 'identity_uid': 'a0-a6',
 'uid': '2019-07-22T15:31:24.874544/fa6efd67',
 'signature': 'bc4uPWcUXmkqQ8wSX3h71plAHrG9WURWs8Zr5FJWVpaijojYwGw9EpyDKAK2UVx5dUmcLUDeBWcx5RZHF32DtYzLssKd2VvktnQvO11Xd4/T0J1OO5iOebb7i8Ju+hM7+G8fv+CPIoYohJSSAXWnEaCVTW56gblzTT8ECs2CXlmVU4tZPeYZWaf9nzoeMBK8HjtnPy2buS1YsU6T4w9meX4G6z7vZZZ1JGWf816kvAFjQvFg+DfhbwpgsFjKbvpGolsYJEPipldGbgJBwJy91s1IOeaCIT844bmwL77rQ57qZBms+hzAjDNUgWjAktCenWGB14IBzySyTVyqrZ5b6Q==',
 'siguid': 'D7pstiu86ypi6cejNLAKwsnn1cS559DMWV9931JmA2kx4oyUz1KnnOn5Xg9Y64HiCTW6YVrSdm2sWDP2nvdB7kAD3cRe/fCOhYX916+2NgvlqBgmf3J5RFdJ1+DLm+NZbP+npN+c0ULrZhkUgNbFJRjVBpTwv1kz2eE1cUiaEnWQdr5edza+N6dSgtiHKx8b2XwuSdJVsi8xyWTnUaBHLmVNacjc4b/LO22cCfr2XTopOG7GQwi4nlLkCPmGIuhi7CvY0AXWZBR4vtyR47DnwH/U4PHQj1bvZNDOfBh0TvMRW+QBX1c3aPZVKZzptrAFLBh7Cvx4dK/DzRMVo8cFXw=='}

In [ ]:
from HUGS.Processing import search

search_terms = widgets.Text(
                    value="Search",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="Locations",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

box = widgets.VBox(children=[search_terms, locations, data_type, search_button], layout=layout)

def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    x =  search(search_terms=search_terms.value, locations=locations.value, data_type=data_type.value)
    print(x)
    



In [ ]:
search_button.on_click(call_search)

In [ ]:
box